[Reference](https://medium.com/better-programming/the-beginners-guide-to-pydantic-ba33b26cde89)

In [1]:
!pip install pydantic

     |████████████████████████████████| 8.7MB 3.7MB/s 


In [2]:
# install email-validator
!pip install email-validator

# install typing-extensions
!pip install typing_extensions

# install python-dotenv
!pip install python-dotenv

     |████████████████████████████████| 215kB 5.9MB/s 


In [3]:
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel

In [4]:
class User(BaseModel):
    id: int
    username : str
    password : str
    confirm_password : str
    alias = 'anonymous'
    timestamp: Optional[datetime] = None
    friends: List[int] = []

In [5]:
data = {'id': '1234', 'username': 'wai foong', 'password': 'Password123', 'confirm_password': 'Password123', 'timestamp': '2020-08-03 10:30', 'friends': [1, '2', b'3']}

user = User(**data)

In [6]:
user

User(id=1234, username='wai foong', password='Password123', confirm_password='Password123', timestamp=datetime.datetime(2020, 8, 3, 10, 30), friends=[1, 2, 3], alias='anonymous')

In [7]:
data = {'id': 'a random string', 'username': 'wai foong', 'password': 'Password123', 'confirm_password': 'Password123', 'timestamp': '2020-08-03 10:30', 'friends': [1, '2', b'3']}

user = User(**data)

ValidationError: ignored

In [8]:
from pydantic import BaseModel, ValidationError

# ... codes for User class

data = {'id': 'a random string', 'username': 'wai foong', 'password': 'Password123', 'confirm_password': 'Password123', 'timestamp': '2020-08-03 10:30', 'friends': [1, '2', b'3']}

try:
    user = User(**data)
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "id"
    ],
    "msg": "value is not a valid integer",
    "type": "type_error.integer"
  }
]


In [9]:
from pydantic import (
    BaseModel,
    NegativeInt,
    PositiveInt,
    conint,
    conlist,
    constr
)

class Model(BaseModel):
    # minimum length of 2 and maximum length of 10
    short_str: constr(min_length=2, max_length=10)    
    
    # regex
    regex_str: constr(regex=r'^apple (pie|tart|sandwich)$')    
    
    # remove whitespace from string
    strip_str: constr(strip_whitespace=True)

    # value must be greater than 1000 and less than 1024
    big_int: conint(gt=1000, lt=1024)
    
    # value is multiple of 5
    mod_int: conint(multiple_of=5)
    
    # must be a positive integer
    pos_int: PositiveInt
    
    # must be a negative integer
    neg_int: NegativeInt

    # list of integers that contains 1 to 4 items
    short_list: conlist(int, min_items=1, max_items=4)

In [11]:
from pydantic import BaseModel, StrictBool

class StrictBoolModel(BaseModel):
    strict_bool: StrictBool

In [12]:
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, ValidationError, validator

class User(BaseModel):
    id: int
    username : str
    password : str
    confirm_password : str
    alias = 'anonymous'
    timestamp: Optional[datetime] = None
    friends: List[int] = []    
    
    @validator('id')
    def id_must_be_4_digits(cls, v):
        if len(str(v)) != 4:
            raise ValueError('must be 4 digits')
        return v    
        
    @validator('confirm_password')
    def passwords_match(cls, v, values, **kwargs):
        if 'password' in values and v != values['password']:
            raise ValueError('passwords do not match')
        return v